In [5]:
import sys
sys.path.append("..")
import json
from pathlib import Path
import scraper
from concurrent.futures import ThreadPoolExecutor, as_completed

In [6]:
data_dir = Path("../data/T20I/")

# Then for each series we can get the match metadata
for year_path in [path for path in data_dir.iterdir() if path.name == "2015"]:
    with ThreadPoolExecutor() as pool:
        futures=[]
        for series_path in year_path.iterdir():
            with open(series_path / "series_meta.json", "r") as f:
                series_meta = json.load(f)

            with open(series_path / "season_meta.json", "r") as f:
                season_meta = json.load(f)
            
            series_id = series_meta["mappings"]["cricinfo"]
            
            event_urls = scraper.extract_items(scraper.json_resp("{}/{}".format(season_meta["$ref"], "events")))
            event_ids = scraper.extract_ids(event_urls)

            matches_dir = series_path / "matches" 

            for match_id, url in zip(event_ids, event_urls):
                match_dir = matches_dir / str(match_id) 
                if (match_dir / "match_meta.json").exists():
                    print("already done")
                else:
                    futures.append(pool.submit(scraper.scrape_and_save, scraper.json_resp, match_dir / "match_meta.json", True, url))
            
        [r.result() for r in as_completed(futures)]
                
                    

Saved data ../data/testing/T20I/2015/11205-afghanistan-v-oman-t20i-series-2015-16/matches/930583/match_meta.json
Saved data ../data/testing/T20I/2015/11205-afghanistan-v-oman-t20i-series-2015-16/matches/930585/match_meta.json
Saved data ../data/testing/T20I/2015/11336-pakistan-in-sri-lanka-t20i-series-2015/matches/860281/match_meta.jsonSaved data ../data/testing/T20I/2015/11336-pakistan-in-sri-lanka-t20i-series-2015/matches/860279/match_meta.json

Saved data ../data/testing/T20I/2015/11362-nepal-in-netherlands-t20i-series-2015/matches/883347/match_meta.jsonSaved data ../data/testing/T20I/2015/11362-nepal-in-netherlands-t20i-series-2015/matches/883341/match_meta.json
Saved data ../data/testing/T20I/2015/11362-nepal-in-netherlands-t20i-series-2015/matches/883343/match_meta.json

Saved data ../data/testing/T20I/2015/11362-nepal-in-netherlands-t20i-series-2015/matches/883345/match_meta.json
Saved data ../data/testing/T20I/2015/11417-pakistan-in-bangladesh-t20i-match-2015/matches/858491/mat